# Sentiment Analysis
In diesem Teil möchten wir uns mit Sentiment Analysis beschäftigen. Vereinfacht gesagt beschäftigt sich Sentiment Analysis damit, natürlichsprachliche Aussagen dahingehend zu bewerten, ob die subjektive Aussage des Sprechers positiv oder negativ wertend gemeint ist.

Zu diesem Zweck haben wir den Datensatz von Sentiment140, einem Projekt der Stanford University, ausgewählt. Er beinhaltet 16 Millionen Tweets, die aufgrund der enthaltenen Emoticons automatisch in positiv und negativ eingeteilt wurden.

Der Datensatz liegt als csv-Datei vor. Zunächst möchten wir uns die darin enthaltene Daten etwas genauer ansehen.

## 1. Aufgabe

1. Laden Sie den Datensatz von http://help.sentiment140.com/for-students
2. Lesen Sie den Datensatz in eine Liste ein, benutzen Sie dazu den csv-reader: https://docs.python.org/3/library/csv.html. Da wir uns nur für die Felder `polarity` und `text` interessieren, sollte die Liste mit den Daten folgendes Format haben : `[(polarity, text),...]`. Positive Tweets haben eine polarity von '4', negative von '0'. Wandeln Sie beim Einlesen diese Werte gleich um in 1 (positiv) und 0 (negativ).

In [10]:
import csv
import re

polarities = []
texts = []

with open('data/training.1600000.processed.noemoticon.csv', 'r', encoding='iso-8859-1') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')
    for row in reader:
        polarities.append(1 if int(row[0]) else 0)
        texts.append(row[-1])
        
data = list(zip(polarities, texts))
print(data[0:10])

[(0, "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"), (0, "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"), (0, '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'), (0, 'my whole body feels itchy and like its on fire '), (0, "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "), (0, '@Kwesidei not the whole crew '), (0, 'Need a hug '), (0, "@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?"), (0, "@Tatiana_K nope they didn't have it "), (0, '@twittera que me muera ? ')]


3. Um einen Einblick in die Daten zu bekommen und um später ein Modell zur Sentiment Analyse trainieren zu können, sollen die Daten zunächst etwas aufbereitet werden. Da die sinntragenden Elemente in den Tweets die Wörter sind, sollten Sie die Tweets in Wörter aufteilen. Um genau zu sein, ist der Term 'Wörter' hier aus linguistischer Sicht etwas falsch, man spricht eigentlich von Tokens. Daher nennt man das Aufteilen von Text auch Tokenizing und die Funktion, die sowas kann, Tokenizer.
Der allereinfachste Tokenizer ist vermutlich die `split` Methode. Tokenisieren Sie damit die eingelesen Tweets. Am Ende sollten Sie eine Liste `tokenized = [(polarity, [token_1,token_2, ...])]` erhalten.

In [11]:
tokens = [text.split() for text in texts]
tokenized = list(zip(polarities, tokens))
print(tokenized[0:10])

[(0, ['@switchfoot', 'http://twitpic.com/2y1zl', '-', 'Awww,', "that's", 'a', 'bummer.', 'You', 'shoulda', 'got', 'David', 'Carr', 'of', 'Third', 'Day', 'to', 'do', 'it.', ';D']), (0, ['is', 'upset', 'that', 'he', "can't", 'update', 'his', 'Facebook', 'by', 'texting', 'it...', 'and', 'might', 'cry', 'as', 'a', 'result', 'School', 'today', 'also.', 'Blah!']), (0, ['@Kenichan', 'I', 'dived', 'many', 'times', 'for', 'the', 'ball.', 'Managed', 'to', 'save', '50%', 'The', 'rest', 'go', 'out', 'of', 'bounds']), (0, ['my', 'whole', 'body', 'feels', 'itchy', 'and', 'like', 'its', 'on', 'fire']), (0, ['@nationwideclass', 'no,', "it's", 'not', 'behaving', 'at', 'all.', "i'm", 'mad.', 'why', 'am', 'i', 'here?', 'because', 'I', "can't", 'see', 'you', 'all', 'over', 'there.']), (0, ['@Kwesidei', 'not', 'the', 'whole', 'crew']), (0, ['Need', 'a', 'hug']), (0, ['@LOLTrish', 'hey', 'long', 'time', 'no', 'see!', 'Yes..', 'Rains', 'a', 'bit', ',only', 'a', 'bit', 'LOL', ',', "I'm", 'fine', 'thanks', ','

4. Abgesehen von natürlichsprachlichen Wörtern sind in Tweets mindestens auch Hashtags, Mentions und Links enthalten. Überlegen Sie sich, ob es Sinn ergibt, alle diese Bestandteile in den Daten in dieser From zu behalten. Begründen Sie kurz Ihre Entscheidungen.
Falls Sie sich entschlossen haben, nicht alle diese Bestandteile zu behalten, filtern Sie dementsprechend Ihre Daten. Die Struktur Ihrer Daten sollte am Ende gleich bleiben: `cleaned = [(polarity, [token_1,...])]`

*to be removed: Links, Mentions* <br/>
*reformatting: everything to lowercase, remove all punctuation*

In [12]:
REGEX_URLS = r'(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%.\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%\+.~#?&//=]*)'
REGEX_PUNCTUATION = r'[^\w\s\']'  # Means everything except word chars (\w), space chars (\s) or apostrophes
REGEX_EMOTICONS = r'(?::|;|=)(?:-)?(?:\)|D|P)'

In [13]:
def filter_token(token):
    '''
    Check if Token is a URL or @mention. If so, return none. Else return token.
    '''
    #remove links
    if re.match(REGEX_URLS, token):
        #print("### Found URL in: ", token)
        return
        
    if "@" in token:
        #print("### Found @ in: ", token)
        return
    
    #print("### Found nothing in: ", token)
    return token

In [14]:
def remove_punctuation_but_keep_emoticons(token):
    '''
    Removes all additional punctuation from a token but keeps it as a whole if it is an emoticon.
    '''
    if re.match(REGEX_EMOTICONS, token):
        #print("### Found emoticon in: ", token)
        return token
    else:
        return re.sub(REGEX_PUNCTUATION, '', token)  # also filters %-signs - do we want this?

In [15]:
tokens_cleaned = [] 


for tokenlistno, tokenlist in enumerate(tokens):
    tokens_cleaned.append([])
    #print("### Tokenlist: ", tokenlist)
    for token in tokenlist:
        #print("### Current token: ", token)
        result = filter_token(token)
        if result:
            result = remove_punctuation_but_keep_emoticons(result)
            if result:   
                result = result.lower()
                #print(result)
                tokens_cleaned[tokenlistno].append(result)


cleaned = list(zip(polarities, tokens_cleaned))
print(cleaned[0:10])

[(0, ['awww', "that's", 'a', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', ';d']), (0, ['is', 'upset', 'that', 'he', "can't", 'update', 'his', 'facebook', 'by', 'texting', 'it', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', 'blah']), (0, ['i', 'dived', 'many', 'times', 'for', 'the', 'ball', 'managed', 'to', 'save', '50', 'the', 'rest', 'go', 'out', 'of', 'bounds']), (0, ['my', 'whole', 'body', 'feels', 'itchy', 'and', 'like', 'its', 'on', 'fire']), (0, ['no', "it's", 'not', 'behaving', 'at', 'all', "i'm", 'mad', 'why', 'am', 'i', 'here', 'because', 'i', "can't", 'see', 'you', 'all', 'over', 'there']), (0, ['not', 'the', 'whole', 'crew']), (0, ['need', 'a', 'hug']), (0, ['hey', 'long', 'time', 'no', 'see', 'yes', 'rains', 'a', 'bit', 'only', 'a', 'bit', 'lol', "i'm", 'fine', 'thanks', "how's", 'you']), (0, ['nope', 'they', "didn't", 'have', 'it']), (0, ['que', 'me', 'muera'])]


5. Zählen Sie die Tokens in Ihrem Datensatz. Benutzen Sie dafür ein Dictionary. Geben Sie die 100 häufigsten Wörter sortiert aus. Was stellen Sie fest? Was müssen Sie zusätzlich noch filtern?

In [16]:
from collections import Counter

tokens_cleaned_flat = [token for tokenlist in tokens_cleaned for token in tokenlist]
#print(tokens_cleaned_flat[0:20])
word_count = Counter(tokens_cleaned_flat)

print("\n\n",list(word_count.items())[0:10])

print("\n\n",word_count.most_common(10)) #word_count_sorted = word_count.most_common()



 [('awww', 5266), ("that's", 21270), ('a', 377409), ('bummer', 1439), ('you', 269785), ('shoulda', 343), ('got', 60733), ('david', 2278), ('carr', 75), ('of', 183404)]


 [('i', 750477), ('to', 564454), ('the', 519495), ('a', 377409), ('my', 313885), ('and', 297875), ('you', 269785), ('is', 235838), ('it', 230657), ('for', 215650)]


## 2. Aufgabe

Wie Eingangs erwähnt, beschäftigt sich Sentiment Analysis damit, eine Äußerung automatisch dahingehend zu klassifizieren,
ob der Inhalt positiv oder negativ gemeint ist.
Im Machine-Learning-Jargon gesprochen hat man es also mit einer binären Klassifikation zu tun. Wir möchten im folgenden Teil ein neuronales Netz trainieren, das entscheiden kann, ob ein Tweet positiv oder negativ gemeint.

Für das Training des neuronalen Netzes möchten wir Keras als Framework benutzen. Keras bietet eine Vereinfachung der Tensorflow-API an, d.h. mit deutlich weniger Aufwand kann man alle Funktionalitäten von Tensorflow benutzen.

Keras bietet zwei unterschiedliche APIs zum Erstellen von neuronalen Netzen an, namentlich _sequential_ und _functional_.
Bei der _sequential_-API wird das Model Schicht für Schicht aufgebaut. Leider kann mit dieser API kein Model aufgebaut werden, das Schichten enthält, die mehr als eine Vorgängerschicht gleichzeitig haben, oder einzelne Schichten wiederbenutzt. Mit der _functional_-API ist dies möglich.

Aktuell liegen unsere Daten zwar in tokenisierter und gesäuberter Form vor, allerdings wird ein neuronales Netz damit sehr wenig anfangen können. Wir müssen unsere Daten also noch etwas weiter vorbereiten.

Als Eingabe soll unser neuronales Netz später Vektoren nehmen, deren einzelne Komponenten alle Wörter darstellen und jeder Eintrag die Anzahl des Wortes in den jeweiligen Tweets. Ein Beispiel:
Zwei Tweets "lorem ipsum" und "foo foo bar", die Vektoren hätten die Länge 4 und für den ersten Tweet wäre der Vektor `[1,1,0,0]`, für den zweiten `[0,0,2,1]`.

1. Befüllen Sie das dictionary `word2idx` so, dass jedes Wort auf einen Index abgebildet wird und die Indizes streng monoton aufsteigend sind. Für das Beispiel oben wäre `word2idx = {"lorem": 0, "ipsum": 1, "foo": 3, "bar": 4}`

In [17]:
word2idx = {word: idx for idx, word in enumerate(word_count.keys())}

print(list(word2idx.items())[0:10])

[('awww', 0), ("that's", 1), ('a', 2), ('bummer', 3), ('you', 4), ('shoulda', 5), ('got', 6), ('david', 7), ('carr', 8), ('of', 9)]


2. Welche Länge werden die Vektoren haben?

In [18]:
VECTOR_LEN = len(word2idx)
print(VECTOR_LEN)

413203


3. Wir könnten mit `numpy` ein Array befüllen, das für jeden der 16 Millionen Tweets einen Vektor wie oben beschrieben enthält. 
Bevor Sie damit beginnen, überschlagen Sie, wieviel Speicherplatz (im Hauptspeicher) ein solches Array belegen würde, wenn jeder Eintrag 32 bit hat. Reicht Ihr Hauptspeicher dafür aus?

In [19]:
MEMORY = 32 * VECTOR_LEN * 16000000
print(MEMORY/8/1000/1000/1000,"GB")

26444.992 GB


4. Um das Problem mit dem zu kleinen Hauptspeicher zu umgehen, bietet Keras die Möglichkeit, anstatt auf einem kompletten Datensatz zu operieren, immer nur kleinere Häppchen abzuarbeiten. Dazu wird ein Python-Generator eingesetzt.
Vervollständigen Sie die Funktion unten, so dass ein Generator entsteht. Die Parameter der Funktion sind:
 * d: tokenisierte und gesäuberte Tweets und Labels
 * w2i: das word2index dictionary
 * batch_size: Anzahl der vektorisierten Tweets, die pro Aufruf zurückgegeben werden sollen.
 
Die benutzen Tweets nacheinander aus `d` gewählt werden und kein Tweet mehrfach zurückgegeben werden.

In [20]:
def bagofwords(tokens, w2i):
    bow = np.zeros((len(w2i)), dtype=np.int)
      
    for token in tokens:
        idx = w2i[token]
        bow[idx] += 1
    
    return bow

In [21]:
import random
import numpy as np

def data_generator(d, w2i, batch_size):
    l = len(d)
    for start_index in range(0, l, batch_size):
        batch = d[start_index:min(start_index+batch_size, l)]
        
        batch_x = np.zeros((batch_size, len(w2i.keys())), dtype=np.int)
        batch_y = np.zeros((batch_size, 1), dtype=np.int)
        # TODO
        for rowno, row in enumerate(batch):
            batch_x[rowno] = bagofwords(row[1], w2i)
            batch_y[rowno] = row[0]
        
        
        yield batch_x, batch_y

Sie können Ihren Generator wie folgt ausprobieren:

In [22]:
gen = data_generator(cleaned, word2idx, 100)

In [23]:
print(next(gen))

(array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]]), array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
 

Wir sind nun endlich soweit, unser neuronales Netz aufzubauen. Da unser Netz genau ein hidden Layer hat und auch sonst nicht sonderlich komplex ist, benutzen wir die _Sequential_-API von Keras.

In [24]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

m = Sequential()

5. Fügen Sie einen _Dense_-Layer dem Netz hinzu, als _hidden units_ können Sie 16 nehmen. Da dies auch der Eingabe-Layer ist, müssen Sie den Parameter `input_shape` definieren. (Siehe auch: https://keras.io/layers/core/)

In [25]:
m.add(Dense(16, input_shape=(VECTOR_LEN,), activation='relu'))

Instructions for updating:
Colocations handled automatically by placer.


6. Als letzten Layer in unserem neuronalen Netz, fügen Sie einen weiteren _Dense_-Layer hinzu. Dieser Layer dient auch als "Ausgabelayer" Überlegen Sie sich die Anzahl der _hidden units_ (Hinweis: Wie lässt sich unser Machine-Learning-Problem kategorisieren?) Welche _Activation_-Funktion wählen Sie?

In [26]:
m.add(Dense(1, activation='sigmoid'))

7. Kompilieren Sie das neuronale Netz. Als `optimizer` können Sie 'adam' benutzen. Wählen Sie eine passende `loss`-Funktion aus. Begründen Sie Ihre Entscheidung. (https://keras.io/models/model/#compile)

In [27]:
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/

8. Bevor Sie nun das neuronale Netz trainieren, teilen Sie noch Ihren Datensatz in zwei Teile auf. Einen Teil zum Trainieren und einen zum Evaluieren. Das Verhältnis der beiden Datensätze sollte 70%:30% sein. Bevor Sie die Daten aufteilen, durchmischen Sie sie mit der `shuffle`-Methode aus dem `random`-Modul. Außerdem sollten Sie die Datenmenge zunächst auf ca. 100000 begrenzen, damit das Training des neuronalen Netzes nicht ewig dauert.

In [28]:
from random import shuffle

cleaned_shuffled = cleaned[:]
shuffle(cleaned_shuffled)

cleaned_train = cleaned_shuffled[:int(len(cleaned_shuffled)*0.7)]
cleaned_eval = cleaned_shuffled[int(len(cleaned_shuffled)*0.7):]

9. Wir sind nun soweit das neuronale Netz zu trainieren. Da wir den oben entwickelten Generator einsetzen wollen, verwenden wir dazu die `fit_generator`-Methode. Als `batch_size` können Sie 100 nehmen, für den `epochs`-Parameter 10. Was wählen Sie als `steps_per_epoch`-Parameter? (https://keras.io/models/model/#fit_generator)

In [34]:
m.fit_generator(data_generator(cleaned_train, word2idx, 100), epochs=10, steps_per_epoch=int(np.ceil(len(cleaned_train) / 1000)))

Epoch 1/10
1120/1120 [==============================] - 194s 174ms/step - loss: 0.4066 - acc: 0.8147
Epoch 2/10
1120/1120 [==============================] - 195s 174ms/step - loss: 0.3944 - acc: 0.8218
Epoch 3/10
1120/1120 [==============================] - 193s 173ms/step - loss: 0.3890 - acc: 0.8239
Epoch 4/10
1120/1120 [==============================] - 193s 172ms/step - loss: 0.3914 - acc: 0.8232
Epoch 5/10
1120/1120 [==============================] - 194s 173ms/step - loss: 0.3867 - acc: 0.8259
Epoch 6/10
1120/1120 [==============================] - 194s 173ms/step - loss: 0.3858 - acc: 0.8267
Epoch 7/10
1120/1120 [==============================] - 194s 173ms/step - loss: 0.3844 - acc: 0.8279
Epoch 8/10
1120/1120 [==============================] - 194s 173ms/step - loss: 0.3840 - acc: 0.8281
Epoch 9/10
1120/1120 [==============================] - 194s 173ms/step - loss: 0.3857 - acc: 0.8276
Epoch 10/10
1120/1120 [==============================] - 194s 173ms/step - loss: 0.3812 - a

### 10. Während das Netz trainiert wird, können Sie sich Gedanken zur Evaluierung machen:
   * Definieren Sie die üblichen Fehlerklassen (wahr positiv, falsch positiv, wahr negativ, falsch negativ)
   * Eine häufig benutzte Evaluationsmetrik ist die _Accuracy_. Beschreiben Sie dieses Metrik und schreiben Sie die Formel zur Berechnung auf.
   * Warum könnte die _Accuracy_ eine schlechte Metrik sein?
   * Zur Evaluation von binären Klassifikationsproblemen wird in der Literatur gerne _Precision_ und _Recall_ verwendet. Wie sind die beiden Evaluationsmaße definiert? Beschreiben Sie diese Metriken mit eigenen Worten. Schreiben Sie auch die Formeln zur Berechnung auf.
   * Warum könnten _Precision_ und _Recall_ bessere Metriken sein als _Accuracy_?

11. Inzwischen sollte das Netz fertig trainiert sein. Speichern Sie es ab!

In [35]:
m.save('my_net_11_epochs_8300.h5')

#### Load saved model

In [39]:
import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    model = load_model('my_net_11_epochs_8300.h5')

Instructions for updating:
Use tf.cast instead.


12. Evaluieren Sie ihr Netz mit dem Datensatz, den Sie oben beseite gelegt haben. Benutzen Sie dafür die `predict_classes`-Methode des Models. Berechnen Sie dafür _Precision_, _Recall_ und _Accuracy_. Interpretieren Sie kurz Ihre Ergebnisse. 

In [45]:
print(len(cleaned_eval[0:100]), len(word2idx.keys()))

cleaned_eval_x = np.zeros((len(cleaned_eval), len(word2idx.keys())), dtype=np.int)
cleaned_eval_y = np.zeros((len(cleaned_eval), 1), dtype=np.int)
        # TODO
for rowno, row in enumerate(cleaned_eval):
    cleaned_eval_x[rowno] = bagofwords(row[1], word2idx)
    cleaned_eval_y[rowno] = row[0]

480000 413203


MemoryError: 

In [41]:
cleaned_eval_predictions = model.predict_classes(eval_batch)

AttributeError: 'generator' object has no attribute 'ndim'

In [ ]:
y_pred = model.predict_generator(data_generator(cleaned_eval, word2idx, 100), steps=int(np.ceil(len(cleaned_eval) / 100))-1, workers=10)

In [ ]:
y_pred.shape

In [ ]:
y_pred[0:10]

In [ ]:
precision = 0.0
recall = 0.0
accuracy = 0.0
#TODO

## Hausaufgabe
1. Trainieren Sie Ihr Netz auf dem großen Datensatz.
2. Verändern Sie die Parameter Ihres Netzes (z.B Anzahl _hidden units_, Anzahl _hidden layers_) und trainieren Sie das Netz erneut (auf dem kleinen Datensatz). Was stellen Sie fest?